# Python with MongoDB

Helpful learning resources: https://www.tutorialspoint.com/mongodb/index.htm

## Libraries and Settings

In [ ]:
# Libraries
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

from pymongo import MongoClient

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(os.getcwd())

## Connect to the MongoDB server and list databases

In [ ]:
# Connect to the MongoDB server
client = MongoClient("mongodb://mongo:27017/")

# List databases
databases = client.list_database_names()
print("Connected to MongoDB. Databases:", databases)

## Read and insert data

In [ ]:
# Create / Access the specific database and collection
db = client['car_database']
collection = db['car_collection']

# Read data from JSON file
with open('/workspace/car_data.json', 'r') as file:
    example_data = json.load(file)

# Insert data into the collection (similar to tables in an SQL database)
try:
    insert_result = collection.insert_many(example_data)
    print(f"Inserted documents to mongo db")
except Exception as e:
    print(f"Insert error: {e}")


## Define a query with multiple conditions

In [ ]:
# Function to query the data
def query_collection(db_name, collection_name, query, client=client):
    if client:
        db = client[db_name]
        collection = db[collection_name]
        results = list(collection.find(query))
        return results
    return []

# Define a query with multiple conditions
query = {
    "Price": {"$gt": 10000, "$lt": 120000},
    "PS": {"$gt": 125},
    "Fuel_Type": {"$eq": 'Benzin'}
}

# Execute the query and fetch the results
results = query_collection('car_database', 'car_collection', query)

# Convert the results into a Pandas DataFrame
df = pd.DataFrame(results)

# Display selected columns
df[['Offer_Id', 'Marke', 'Type', 'Fuel_Type', 'Price', 'PS']].head()

## Create histogram of car prices

In [ ]:
# Create a histogram of car prices
plt.hist(df['Price'], bins=50, color='greenyellow')
plt.title('Histogram of Car Prices')
plt.xlabel('Car price (CHF)')
plt.ylabel('Frequency')
plt.grid()
plt.show()

## Define a query to aggregate the data

In [ ]:
# Funktion to aggregate data
def aggregate_prices_by_brand(db_name, collection_name, client=client):
    db = client[db_name]
    collection = db[collection_name]
    
    pipeline = [
        {
            "$group": {
                "_id": "$Marke",
                "totalPrice": { "$sum": "$Price" },
                "averagePrice": { "$avg": "$Price" },
                "count": { "$sum": 1 }
            }
        },
        {
            "$sort": { "totalPrice": -1 }
        }
    ]
    
    results = list(collection.aggregate(pipeline))
    return results

# Execute the aggregation query
results = aggregate_prices_by_brand('car_database', 'car_collection')

# Convert the results into a Pandas DataFrame
df = pd.DataFrame(results)

# Rename the '_id' column to 'Marke' for better readability
df.rename(columns={'_id': 'Marke'}, inplace=True)

# Display the DataFrame
df.head()

## Create bar chart of average car prices

In [ ]:
#  Subset with the 20 most important brands in terms of the occurrence of vehicles
df_sub = df.head(20).sort_values(by='averagePrice')

# Create a bar chart
plt.figure(figsize=(6, 5))
plt.barh(df_sub["Marke"], df_sub["averagePrice"], color='greenyellow')
plt.title('Average Prices by Marke')
plt.xlabel('Average Price')
plt.ylabel('Marke')
plt.grid(axis='x')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Display the plot
plt.show()

## Drop defined collections and databases

In [ ]:
# List databases
databases = client.list_database_names()
print("Connected to MongoDB. Databases:", databases)

# Drop defined collections and database if exists
db_name = 'car_database'
collection_name = 'car_collection'
db = client[db_name]
if collection_name in db.list_collection_names():
    db.drop_collection(collection_name)
    print(f"Collection '{collection_name}' dropped from database '{db_name}'.")
else:
    print(f"Collection '{collection_name}' does not exist in database '{db_name}'.")

# Drop the database
if db_name in databases:
    client.drop_database(db_name)
    print(f"Database '{db_name}' dropped.")
else:
    print(f"Database '{db_name}' does not exist.")


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')